In [1]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint

In [13]:
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
sparql.setQuery('''
SELECT ?property ?value
WHERE {
  dbr:Matrix ?property ?value.
}
''')
sparql.setReturnFormat(JSON)
qres = sparql.query().convert()

# pprint(qres)
for result in qres['results']['bindings']:
    # print(result['object'])
    
    lang, value = result['property'], result['value']
    print(f'Property: {lang}\tValue: {value}')
    # if lang == 'en':
        # print(value)


Property: {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}	Value: {'type': 'literal', 'xml:lang': 'en', 'value': 'Matrix'}
Property: {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}	Value: {'type': 'literal', 'xml:lang': 'ar', 'value': 'ماتريكس (توضيح)'}
Property: {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}	Value: {'type': 'literal', 'xml:lang': 'ca', 'value': 'Matriu'}
Property: {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}	Value: {'type': 'literal', 'xml:lang': 'cs', 'value': 'Matrix (rozcestník)'}
Property: {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}	Value: {'type': 'literal', 'xml:lang': 'de', 'value': 'Matrix'}
Property: {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}	Value: {'type': 'literal', 'xml:lang': 'es', 'value': 'Matrix (desambiguación)'}
Property: {'type': 'uri', 'value': 'http://www.w3.org/2000/01/rdf-schema#label'}	Value: {'t

In [3]:
query = """
SELECT ?property ?value
WHERE {
  dbr:Matrix ?property ?value.
}
"""

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Print the results
for result in results["results"]["bindings"]:
    print(f"Country: {result['country']['value']}")
    print(f"Capital: {result['capital']['value']}")
    print(f"Population: {result['population']['value']}\n")


In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [7]:


def get_related_films(wikidata_id):
  endpoint_url = "https://query.wikidata.org/sparql"
    
  # SPARQL query to get films based on the specified rules
  query = f"""
  SELECT DISTINCT ?z
  WHERE {{
      {{
        ?x wdt:P136 ?genre.
        ?z wdt:P136 ?genre.
        
        ?x wdt:P577 ?publicationDateX.
        ?z wdt:P577 ?publicationDateZ.
        FILTER(?publicationDateZ >= ?publicationDateX - 10 && ?publicationDateZ <= ?publicationDateX + 10)
        FILTER(?x != ?z)
      }}
      UNION
      {{
        ?x wdt:P57 ?director.
        ?z wdt:P57 ?director.
        
        FILTER(?x != ?z)
      }}
      UNION
      {{
        ?x wdt:P921 ?mainSubject.
        ?z wdt:P921 ?mainSubject.
        
        FILTER(?publicationDateZ >= ?publicationDateX - 10 && ?publicationDateZ <= ?publicationDateX + 10)
        FILTER(?x != ?z)
      }}

    VALUES ?x {{ wd:{wikidata_id} }}
  }}
  """

    
  # Set up SPARQL query and execute it
  sparql = SPARQLWrapper(endpoint_url)
  sparql.setQuery(query)
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  
  # Extract and return the list of related film IDs
  related_films = [result['z']['value'].split('/')[-1] for result in results['results']['bindings']]
  return related_films

# Example usage
wikidata_id = "Q83495"  # Replace with the actual Wikidata film ID
result = get_related_films(wikidata_id)
print(result)


['Q189600', 'Q207536', 'Q312078', 'Q736528', 'Q864930', 'Q959892', 'Q1203242', 'Q1210827', 'Q12125299', 'Q17080059', 'Q80322391', 'Q28936']


In [8]:
len(result)

12

In [6]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_main_subjects(film_ids):
    endpoint_url = "https://query.wikidata.org/sparql"
    
    main_subjects = []
    
    for film_id in film_ids:
        # SPARQL query to get main subjects of a film
        query = f"""
        SELECT DISTINCT ?mainSubjectLabel
        WHERE {{
          wd:{film_id} wdt:P921 ?mainSubject.
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        """
        
        # Set up SPARQL query and execute it
        sparql = SPARQLWrapper(endpoint_url)
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        
        # Extract main subjects and add them to the main_subjects list
        subjects = {
            "film_id":film_id, 
            "main_subjects": [result['mainSubjectLabel']['value'] for result in results['results']['bindings']]
        }
        main_subjects.append(subjects)
    
    return main_subjects

# Example usage:  # Example film IDs
main_subjects = get_main_subjects(result)
print(main_subjects)


[{'film_id': 'Q189600', 'main_subjects': ['telepresence']}, {'film_id': 'Q207536', 'main_subjects': ['telepresence']}, {'film_id': 'Q312078', 'main_subjects': ['motor car']}, {'film_id': 'Q736528', 'main_subjects': []}, {'film_id': 'Q864930', 'main_subjects': ['organized crime']}, {'film_id': 'Q959892', 'main_subjects': []}, {'film_id': 'Q1203242', 'main_subjects': []}, {'film_id': 'Q1210827', 'main_subjects': ['artificial intelligence']}, {'film_id': 'Q12125299', 'main_subjects': []}, {'film_id': 'Q17080059', 'main_subjects': ['connection', 'LGBT']}, {'film_id': 'Q80322391', 'main_subjects': ['free will', 'virtual reality', 'love', 'actuality', 'human-machine relationship', 'dream and reality', 'autonomy', 'human nature']}, {'film_id': 'Q28936', 'main_subjects': ['cloning', 'slavery', 'seamanship']}]


In [ ]:
# Query to link film from dbpedia to wikidata
"""PREFIX wd: <http://www.wikidata.org/entity/>

SELECT DISTINCT ?film
WHERE {
  ?film rdf:type dbo:Film ;
        owl:sameAs wd:Q83495.
}
LIMIT 100"""